In [15]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Define paths to dataset
data_dir = "color"  # Replace with your dataset path

# Parameters
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 20

# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Training and validation generators
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

# Save Model
model.save("plant_disease_model.h5")

# Plot Training History
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

    plt.show()

plot_training_history(history)


Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.
Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.2767 - loss: 2.7423

Time to Test with an Image 

In [3]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("plant_disease_model.h5")


In [4]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def preprocess_image(image_path, target_size=(128, 128)):
    # Load the image
    img = load_img(image_path, target_size=target_size)
    # Convert to array and normalize
    img_array = img_to_array(img) / 255.0
    # Add batch dimension
    return np.expand_dims(img_array, axis=0)


In [14]:
# Class indices (from training generator)
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}  # Reverse mapping

# Test with a sample image
image_path = "0a0d6a11-ddd6-4dac-8469-d5f65af5afca___RS_HL 0555.jpg"  # Replace with your test image path
processed_image = preprocess_image(image_path)

# Predict
prediction = model.predict(processed_image)
predicted_class = np.argmax(prediction, axis=1)[0]
class_name = class_labels[predicted_class]

print(f"The model predicts: {class_name}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The model predicts: Tomato___healthy


In [10]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
    "color",  # Replace with your test dataset path
    target_size=(128, 128),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Evaluate model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")

Found 54305 images belonging to 38 classes.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1698/1698 ━━━━━━━━━━━━━━━━━━━━ 60s 35ms/step - accuracy: 0.9435 - loss: 0.1823
Test Accuracy: 0.93
